In [83]:
from elasticsearch import Elasticsearch
elastic_client = Elasticsearch(hosts=["http://143.167.8.152:9300"], http_auth=('students', 'foxtrot'))

def tweets_in_range(startdate, enddate, amount):    
    
    
    query_body = {
      "query": {
        "bool": {
          "must": {
            "range": {
              "tweet_created_at": {
                "gte": datetime.datetime.strptime(startdate, "%Y-%m-%dT%H:%M:%S.%fZ"),
                "lte":  datetime.datetime.strptime(enddate, "%Y-%m-%dT%H:%M:%S.%fZ"),
                "boost": 2.0
              }
            }
          },
          "must_not": [
            {"match": {"hashtags": "obamagate"}},
            {"match": {"message": "*media_url*"}}
          ]
        }
      },
      "collapse" : {
        "field" : "tweet_id.keyword" 
      },
      "sort": ["tweet_created_at"] 
    }
    
    result = elastic_client.search(index="covid19_misinfo_index", body=query_body, size=amount, request_timeout=30000, _source="tweet_text,tweet_id")

    all_hits = result['hits']['hits']
    
    tweets = []
    for num, doc in enumerate(all_hits):
        tweets.append([doc["_source"]["tweet_text"], doc["_source"]["tweet_id"]])
    return tweets    

In [85]:
print(tweets_in_range("2020-09-10T00:10:57.000Z", "2020-12-10T00:10:59.000Z", 10))

[['RT @kadajoza: @EvanAKilgore @realDonaldTrump I think he knew the truth if it - has always known the truth of it - that it was a #PlanDemic…', '1303848403441344512'], ['RT @TomFitton: #Hydroxychloroquine is a safe drug.', '1303848413029715969'], ['RT @KatherineWolfr2: I agree,and I’m so tired of this crap &amp; really tired of trying to be nice about it..I absolutely believe this was orga…', '1303848431522385920'], ['RT @TomFitton: #Hydroxychloroquine is a safe drug.', '1303848435997581312'], ['RT @TomFitton: #Hydroxychloroquine is a safe drug.', '1303848503718932480'], ["And let's NOT forget about Trump drug pushing &amp; stockpiling #Hydroxychloroquine &amp; making people like myself with Lupus and autoimmune disease (who DEPEND on this medication daily to live) ration the drug &amp; go without! I was 1 of those people.\n\n#TrumpKnewVoteBlue", '1303848517778264064'], ['RT @TomFitton: #Hydroxychloroquine is a safe drug.', '1303848523461308416'], ['RT @TomFitton: #Hydroxychloroquine 